In [2]:
from IPython.display import display, HTML
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as pyfunc
import pyspark.sql.types as pytype
display(HTML('<style>pre { white-space: pre !important; }</style>'))
from pyspark.sql.functions import when
from pyspark.sql.functions import col
from pyspark.sql.functions import expr
from pyspark.sql.functions import sum,avg,max,count
from pyspark.sql.functions import desc, sum, asc


In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=f1e840dc6e464b2c619435a24d1046eaca918865bae9ba7835371913ec2b7a3d
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [3]:
spark = SparkSession.builder.getOrCreate()
spotify_df = spark.read.parquet("/content/drive/MyDrive/files/spotify.parquet")
spotify_df.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- album: string (nullable = true)
 |-- album_id: string (nullable = true)
 |-- artists: string (nullable = true)
 |-- artist_ids: string (nullable = true)
 |-- track_number: long (nullable = true)
 |-- disc_number: long (nullable = true)
 |-- explicit: boolean (nullable = true)
 |-- danceability: double (nullable = true)
 |-- energy: double (nullable = true)
 |-- key: long (nullable = true)
 |-- loudness: double (nullable = true)
 |-- mode: long (nullable = true)
 |-- speechiness: double (nullable = true)
 |-- acousticness: double (nullable = true)
 |-- instrumentalness: double (nullable = true)
 |-- liveness: double (nullable = true)
 |-- valence: double (nullable = true)
 |-- tempo: double (nullable = true)
 |-- duration_ms: long (nullable = true)
 |-- time_signature: double (nullable = true)
 |-- year: long (nullable = true)
 |-- release_date: string (nullable = true)



In [ ]:
spotify_df.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+-----------+--------+------------------+------------------+---+-------------------+----+-----------+-----------------+--------------------+------------------+------------------+------------------+-----------+--------------+----+------------+
|                  id|                name|               album|            album_id|             artists|          artist_ids|track_number|disc_number|explicit|      danceability|            energy|key|           loudness|mode|speechiness|     acousticness|    instrumentalness|          liveness|           valence|             tempo|duration_ms|time_signature|year|release_date|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+-----------+--------+------------------+------------------+---+-------------------+----+--------

In [ ]:
spotify_df.select('disc_number').distinct().collect()

[Row(disc_number=7),
 Row(disc_number=6),
 Row(disc_number=9),
 Row(disc_number=5),
 Row(disc_number=1),
 Row(disc_number=10),
 Row(disc_number=3),
 Row(disc_number=12),
 Row(disc_number=8),
 Row(disc_number=11),
 Row(disc_number=2),
 Row(disc_number=4),
 Row(disc_number=13)]

In [ ]:
df = spotify_df.withColumn("release_date", pyfunc.to_date(spotify_df.release_date, "yyyy-MM-dd"))
df = df.withColumn("artists", pyfunc.from_json(pyfunc.col("artists"), pytype.ArrayType(pytype.StringType())))
df = df.withColumn("artist_ids", pyfunc.from_json(pyfunc.col("artist_ids"), pytype.ArrayType(pytype.StringType())))
# df = df.drop('year')
df.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- album: string (nullable = true)
 |-- album_id: string (nullable = true)
 |-- artists: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- artist_ids: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- track_number: long (nullable = true)
 |-- disc_number: long (nullable = true)
 |-- explicit: boolean (nullable = true)
 |-- danceability: double (nullable = true)
 |-- energy: double (nullable = true)
 |-- key: long (nullable = true)
 |-- loudness: double (nullable = true)
 |-- mode: long (nullable = true)
 |-- speechiness: double (nullable = true)
 |-- acousticness: double (nullable = true)
 |-- instrumentalness: double (nullable = true)
 |-- liveness: double (nullable = true)
 |-- valence: double (nullable = true)
 |-- tempo: double (nullable = true)
 |-- duration_ms: long (nullable = true)
 |-- time_signature: double (nullable = true)
 |-- year: long (nulla

In [ ]:
first_row = df.filter(df.year == 2014).first()
print(first_row['artists'][0])
print(first_row['artists'][1])

Mauricio Kagel
Ensemble Musikfabrik


In [ ]:
df.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+-----------+--------+------------------+------------------+---+-------------------+----+-----------+-----------------+--------------------+------------------+------------------+------------------+-----------+--------------+----+------------+
|                  id|                name|               album|            album_id|             artists|          artist_ids|track_number|disc_number|explicit|      danceability|            energy|key|           loudness|mode|speechiness|     acousticness|    instrumentalness|          liveness|           valence|             tempo|duration_ms|time_signature|year|release_date|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+-----------+--------+------------------+------------------+---+-------------------+----+--------

In [ ]:
first_row = df.first()
print(first_row['artists'])

['Rage Against The Machine']


In [ ]:
first_row = spotify_df.filter(df.year == 2014).first()
print(first_row['artists'])

['Mauricio Kagel', 'Ensemble Musikfabrik']


In [ ]:
df = df.where(df.year>=1800)

In [ ]:
df = df.withColumn("decade",expr("year - year % 10 + 10"))

In [ ]:
df = df.withColumn("explicit_stat",when(df["explicit"],1).otherwise(0))

In [ ]:
def m(df,col):
  return df.withColumn(str(col)+"_label",when((df[col] >= 0) &( df[col] <0.1) ,"0-0.1")
.when((df[col] >= 0.1) & (df[col] <0.2) ,"0.1-0.2")
.when((df[col] >= 0.2) & (df[col] <0.3) ,"0.2-0.3")
.when((df[col] >= 0.3) & (df[col] <0.4) ,"0.3-0.4")
.when((df[col] >= 0.4) & (df[col] <0.5) ,"0.4-0.5")
.when((df[col] >= 0.5) & (df[col] <0.6) ,"0.5-0.6")
.when((df[col] >= 0.6) & (df[col] <0.7) ,"0.6-0.7")
.when((df[col] >= 0.7) & (df[col] <0.8) ,"0.7-0.8")
.when((df[col] >= 0.8) & (df[col] <0.9) ,"0.8-0.9")
.otherwise("0.9-1"))

In [ ]:
def conv_to_min(df):
  return df.withColumn("duration_min",df.duration_ms/60000)

In [ ]:
df = df.transform(conv_to_min)
df = df.transform(m,"danceability")\
.transform(m,"energy")\
.transform(m,"speechiness")\
.transform(m,"acousticness")\
.transform(m,"instrumentalness")\
.transform(m,"liveness")\
.transform(m,'valence')

In [ ]:
#Larghissimo (بسیار بسیار آهسته): کمتر از ۲۰ ضرب در دقیقه.
#Grave (بسیار آهسته): ۲۰ تا ۴۰ ضرب در دقیقه.
#Largo (آهسته و گسترده): ۴۰ تا ۶۰ ضرب در دقیقه.
#Adagio (کند): ۶۰ تا ۷۶ ضرب در دقیقه.
#Andante (کند معتدل): ۷۶ تا ۱۰۸ ضرب در دقیقه.
#Moderato (معتدل): ۱۰۸ تا ۱۲۰ ضرب در دقیقه.
#Allegro (تند): ۱۲۰ تا ۱۶۸ ضرب در دقیقه.
#Presto (بسیار تند): بیش از ۱۶۸ ضرب در دقیقه
df = df.withColumn("tempo_label",when(df["tempo"] < 20,"Larghissimo")
.when((df["tempo"] >= 20)&(df["tempo"] < 40),"Grave")
.when((df["tempo"] >= 40)&(df["tempo"] < 60),"Largo")
.when((df["tempo"] >= 60)&(df["tempo"] < 76),"Adagio")
.when((df["tempo"] >= 76)&(df["tempo"] < 108),"Andante")
.when((df["tempo"] >= 108)&(df["tempo"] < 120),"Moderato")
.when((df["tempo"] >= 120)&(df["tempo"] < 168),"Allegro")
.otherwise("Presto")
)

In [ ]:
df.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+-----------+--------+------------------+------------------+---+-------------------+----+-----------+-----------------+--------------------+------------------+------------------+------------------+-----------+--------------+----+------------+------+-------------+------------------+------------------+------------+-----------------+------------------+----------------------+--------------+-------------+-----------+
|                  id|                name|               album|            album_id|             artists|          artist_ids|track_number|disc_number|explicit|      danceability|            energy|key|           loudness|mode|speechiness|     acousticness|    instrumentalness|          liveness|           valence|             tempo|duration_ms|time_signature|year|release_date|decade|explicit_stat|      duration_min|danceability_label|energ

In [ ]:
df.groupBy("decade") \
    .agg(count("*").alias("count"),\
    avg("explicit_stat").alias("explicit_rate"),\
    avg("danceability").alias("danceability_rate"),\
    avg("loudness").alias("loudness_rate"),\
    avg("acousticness").alias("acousticness_rate"),\
    avg("valence").alias("valence_rate"),\
    )\
    .sort(asc("decade"))\
    .show(truncate=False)

+------+------+---------------------+-------------------+-------------------+-------------------+-------------------+
|decade|count |explicit_rate        |danceability_rate  |loudness_rate      |acousticness_rate  |valence_rate       |
+------+------+---------------------+-------------------+-------------------+-------------------+-------------------+
|1910  |58    |0.0                  |0.5808793103448276 |-14.754137931034485|0.7028051724137931 |0.6722448275862069 |
|1920  |52    |0.0                  |0.5069423076923076 |-16.892826923076928|0.8109284615384618 |0.48089230769230773|
|1930  |461   |0.0                  |0.6124967462039039 |-13.351028199566167|0.9500889370932765 |0.627997613882863  |
|1940  |453   |0.0                  |0.5720066225165568 |-13.777507726269317|0.8476748335540841 |0.5724106181015447 |
|1950  |653   |0.0                  |0.41908728943338447|-14.964433384379793|0.8948390505359883 |0.40191163859111795|
|1960  |3159  |0.0                  |0.4265659069325732 

In [ ]:
df.groupBy("explicit_stat") \
    .agg(count("*").alias("count"),\
    avg("danceability").alias("danceability_rate"),\
    avg("energy").alias("energy_rate"),\
    avg("loudness").alias("loudness_rate"),\
    avg("speechiness").alias("speechiness_rate"),\
    avg("acousticness").alias("acousticness_rate"),\
    avg("instrumentalness").alias("instrumentalness_rate"),\
    )\
    .sort(asc("explicit_stat"))\
    .show(truncate=False)

+-------------+-------+------------------+------------------+-------------------+-------------------+-------------------+---------------------+
|explicit_stat|count  |danceability_rate |energy_rate       |loudness_rate      |speechiness_rate   |acousticness_rate  |instrumentalness_rate|
+-------------+-------+------------------+------------------+-------------------+-------------------+-------------------+---------------------+
|0            |1121367|0.4830483240099053|0.4965064366940072|-12.117940979179641|0.07377358206547045|0.46580304277687556|0.2993331429687508   |
|1            |82648  |0.6288146561320302|0.6863260764398469|-7.612601139773397 |0.22831835737102102|0.18829414126052443|0.05931239955727969  |
+-------------+-------+------------------+------------------+-------------------+-------------------+-------------------+---------------------+



In [ ]:
df.groupBy("liveness_label") \
    .agg(count("*").alias("count"),\
    avg("energy").alias("energy_rate"),\
    avg("loudness").alias("loudness_rate"),\
    avg("speechiness").alias("speechiness_rate"),\
    avg("instrumentalness").alias("instrumentalness_rate"),\
    )\
    .sort(asc("liveness_label"))\
    .show(truncate=False)

+--------------+------+-------------------+-------------------+-------------------+---------------------+
|liveness_label|count |energy_rate        |loudness_rate      |speechiness_rate   |instrumentalness_rate|
+--------------+------+-------------------+-------------------+-------------------+---------------------+
|0-0.1         |334846|0.47248503496860994|-12.753904209696293|0.07167325367482556|0.32054288386636326  |
|0.1-0.2       |502041|0.4532486739274314 |-12.547020014301529|0.07290533263219814|0.3111585501831036   |
|0.2-0.3       |133221|0.5846215666268859 |-10.194569054428445|0.09011841226232988|0.2226870813823613   |
|0.3-0.4       |115152|0.64715324052471   |-9.365629420244524 |0.1035335469640104 |0.21076962729670184  |
|0.4-0.5       |29951 |0.6489240573603482 |-9.662788454475702 |0.13072165203165143|0.2083070309555608   |
|0.5-0.6       |18991 |0.6257604850508103 |-10.169823705965971|0.13807168658838384|0.2118642719451323   |
|0.6-0.7       |24302 |0.5847604744136286 |-11

In [ ]:
df.groupBy("valence_label") \
    .agg(count("*").alias("count"),\
    avg("explicit_stat").alias("explicit_rate"),\
    avg("danceability").alias("danceability_rate"),\
    avg("energy").alias("energy_rate"),\
    avg("loudness").alias("loudness_rate"),\
    avg("acousticness").alias("acousticness_rate"),\
    avg("instrumentalness").alias("instrumentalness_rate"),\
    avg("tempo").alias("tempo_rate"),\
    avg("duration_min").alias("duration_rate")\
    )\
    .sort(asc("valence_label"))\
    .show(truncate=False)

+-------------+------+--------------------+------------------+------------------+-------------------+-------------------+---------------------+------------------+------------------+
|valence_label|count |explicit_rate       |danceability_rate |energy_rate       |loudness_rate      |acousticness_rate  |instrumentalness_rate|tempo_rate        |duration_rate     |
+-------------+------+--------------------+------------------+------------------+-------------------+-------------------+---------------------+------------------+------------------+
|0-0.1        |163934|0.024637964058706552|0.2815761580880098|0.2798768438377655|-18.470007478619337|0.6765733968206166 |0.56181748011364     |104.4551918393987 |5.225780775698337 |
|0.1-0.2      |149806|0.04574583127511582 |0.398996239135946 |0.388160953907722 |-14.120375418875065|0.5639718372563944 |0.3809823172881542   |113.75408188590403|4.565502217868445 |
|0.2-0.3      |137781|0.062171126643005926|0.453772450120114 |0.459611275321716 |-12.16742

In [4]:
K = 10
top_k_records = spotify_df.orderBy(spotify_df.danceability.desc()).limit(K)
top_k_records.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+-----------+--------+------------+------------------+---+-------------------+----+------------------+------------------+--------------------+------------------+------------------+-------+-----------+--------------+----+------------+
|                  id|                name|               album|            album_id|             artists|          artist_ids|track_number|disc_number|explicit|danceability|            energy|key|           loudness|mode|       speechiness|      acousticness|    instrumentalness|          liveness|           valence|  tempo|duration_ms|time_signature|year|release_date|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+------------+-----------+--------+------------+------------------+---+-------------------+----+------------------+-------------